# MA-MOROB-M02: Trajectory Estimation (Semester 1)

*part of the Programme "Mobile Robotics" (M.Sc.) at Bonn University*

## Methods for high precision trajectory estimation of mobile mapping vehicles on crop fields

## The Need for High Precision Trajectory Estimation (in Agricultural Mapping)


<div align="center">
    <img src="./assets/trajectory-need-illustration.svg" width="40%"/>
</div>

- **Positional Accuracy**: Even small deviations from planned trajectories can lead to significant consequences:
  - **Overlapping treatment**: Wasting resources (fertilizer, pesticides, seeds)
  - **Missed areas**: Reducing yield and creating uneven crop development
  - **Crop damage**: Physical damage from vehicles moving off designated paths

- **Mapping Quality**: Poor trajectory estimation directly impacts the quality of collected data, affecting:
  - Precise crop monitoring (e.g. tracking individual crops over time)
  - Accurate field mapping
  - Reliable yield prediction models


## A Concrete Problem Introduction

<!--
* Add robot data recording video here
* explain to want to come back multiple times, monitor crop growth
* introduce the problem of robot localisation
* ask about other scenarios where precision localisation is needed
* explain the simplicity of a row (1D) in the strawberry tunnel case
-->

<div align="middle">
<video width="80%" controls autoplay loop muted>
      <source src="assets/thorvald-data.mp4" type="video/mp4">
</video></div>

<center>An Example of Crop Monitoring with a Mobile Robot</center>

**Strawberry Producer: "I want to monitor the growth of my individual strawberry plants, identifying underperforming ones"**


## A Crop Monitoring Robot


<table>
    <tr>
        <td width="40%">
            <img src="./assets/thorvald-cameras.jpg" width="100%"/>
        </td>
        <td>

### Robot Specifications
* 4 wheel drive, 4 wheel steering
* Multiple Cameras, including Hyperspectral
* LiDAR
* GPS
* IMU
* Odometry

### Task
* Monitor crop growth
* Identify underperforming plants
* Create yield prediction models
* Disease detection (powdery mildew)

</td></tr></table>


## Module "Trajectory Estimation" -- Learning Objectives

1. **Understand and Apply**: Demonstrate a comprehensive understanding of theoretical principles underlying trajectory estimation, including coordinate systems, sensor models, and state estimation techniques.
2. **Analyse and Evaluate**: Critically analyse various trajectory estimation methods and evaluate their suitability for different environments and applications.
3. **Design and Implement**: Design and implement appropriate sensor fusion algorithms for accurate trajectory estimation in challenging environments.
4. **Evaluate Performance**: Assess the performance of trajectory estimation systems using appropriate metrics and validation techniques.
5. **Problem Solve**: Develop solutions for trajectory estimation in environments with specific challenges, such as GPS-denied areas, dynamic obstacles, or varying terrain.
6. **Research and Innovate**: Demonstrate awareness of current research trends and innovations in trajectory estimation for mobile sensing platforms.


## Draft Module Syllabus

### Fundamentals of Trajectory Estimation and Problem Introduction
- Coordinate systems and transformations (global vs. local)
- Introduction to state representation and rigid body kinematics
- Applications and challenges in trajectory estimation (example Crop Monitoring and Precision Agriculture)
- Precision vs. accuracy in different application domains



### Global Localisation for Trajectory Estimation
- GNSS/GPS principles and error sources
- RTK and PPK techniques for high-precision positioning
- GNSS limitations in various environments

### Local Sensors for Trajectory Estimation
- Interoception: 
  - Inertial Measurement Units (IMUs): principles and types
  - Odometry sensors (wheel encoders, visual odometry)
- Exteroception: Environmental sensors (LiDAR, cameras, radar)



### Sensor Fusion & Estimation Theory
- Probabilistic state estimation fundamentals
- Kalman filtering: theory and implementation
- Kalman Filter variants (EKF, UKF)
- Particle filters and Monte Carlo methods
- Smoothing techniques vs. filtering techniques

### Multi-Sensor Calibration
- Calibration of sensor arrays
- Online calibration techniques
- Sensor degradation detection and handling
- Quality metrics and validation approaches



### Advanced Topics in Trajectory Estimation
- Deep learning approaches for state estimation
- Place recognition and loop closure techniques
- Long-term autonomy 
- Distributed (multi-agent) trajectory estimation

### Real World Applications and Limitations
- Real-time implementation considerations and computational complexity
- Methods for GPS-denied environments
- Dynamic (slow and fast) changing environments
- Example Applications
    - Infrastructure inspection
    - Autonomous agricultural vehicles and precision farming
    - Indoor mobile robots and warehouse automation



## Assessment

* Oral Examination 50%
    - 20 minutes oral examination on the theoretical principles of trajectory estimation
    - indicative topics: coordinate systems, sensor models, state estimation techniques, sensor fusion, calibration, and validation
* Coursework, 50%
    - Handling different real-world sensor data sets (IMU, LiDAR, camera, GPS)
    - Implementing a selection of trajectory estimation algorithms
    - Comparative scientific evaluation of their performance
    - Implementing a trajectory estimation system in ROS2 (simulation in devcontainer)
    - Presentation of results and discussion of findings (assessed)
<div align="middle">
<img width="40%" src="./assets/gazebo.png">
</div>




## Foundations: Mathematical Representation of a 3D Trajectory

**A robot trajectory refers to the path that a rigid body (robot) follows through space over time**


**Includes both the position and orientation**


<div align="middle">
<img width="80%" src="./assets/trajectory-illustration.svg">
</div>

## Special Euclidean Group SE(3)

The most comprehensive representation of any rigid body trajectory is through the Special Euclidean Group in 3 dimensions, denoted as $SE(3)$. 

This representation captures both the position (translation) and orientation (rotation) of a rigid body in 3D space.

Mathematically, SE(3) can be represented as a 4×4 matrix:

$$
\begin{pmatrix} 
R & \vec{t} \\
0 & 1 
\end{pmatrix}
$$

Where:
- $R$ is a 3×3 rotation matrix that belongs to the Special Orthogonal Group SO(3)
- $\vec{t}$ is a 3×1 translation vector
- The bottom row [0 0 0 1] completes the homogeneous transformation



# Some Properties of the Special Euclidean Group SE(3)

- **Rigid Body Transformations**: SE(3) represents all possible rigid body transformations (rotations and translations) in 3D space
  
- **Structure**:
  - 6 degrees of freedom (3 for rotation, 3 for translation)
  
- **Key Properties**:
  - **Group Structure**: Closed under composition (combining two transformations yields another valid transformation)
  - **Invertibility**: Every transformation has an inverse
  - **Non-commutativity**: Order matters ($T_1 \cdot T_2 \neq T_2 \cdot T_1$)

## Relevance for Robotics more generally

* This representation allows us to transform points from one coordinate frame to another
* In Robotics often need to relate positions between different reference frames (e.g., robot base frame, world frame, end-effector frame)
    * trajectory estimation
    * calibration

<img src="./assets/frames.svg" width="100%"/>




## Time Parameterisation

A complete trajectory representation includes time as a parameter. This can be written as:

$$T(t) \in SE(3)$$

Where $T(t)$ represents the pose (position and orientation) of the robot at time $t$.

In practical robotics applications, trajectories are often represented as:

1. **Discrete Waypoints**: A sequence of poses $(T_1, T_2, ..., T_n)$ that the robot should pass through at specific times $(t_1, t_2, ..., t_n)$.
2. **Parametric Curves**: Such as splines or polynomials that smoothly interpolate between waypoints.
3. **Joint Space Trajectories**: For robot manipulators, trajectories are often represented in joint space rather than Cartesian space, as a time-varying vector of joint angles $θ(t)$.

<div align="middle">
<img width="40%" src="./assets/trajectory-illustration.svg">
</div>


## in 2D

### Special Euclidean Group SE(2)

* Captures both position and orientation in a plane.
* SE(2) can be represented as a 3×3 matrix of the form:

$$
\begin{pmatrix} 
\cos(\theta) & -\sin(\theta) & t_x \\
\sin(\theta) & \cos(\theta) & t_y \\
0 & 0 & 1 
\end{pmatrix}
$$

Where:
- The 2×2 rotation matrix in the upper-left represents rotation by angle θ in the plane
- $(t_x, t_y)$ is the 2D translation vector
- The bottom row [0 0 1] completes the homogeneous transformation


A minimal representation using $(x, y, \theta)$ where $(x, y)$ is the position and $\theta$ is the orientation angle.




### Specific 2D Applications

Several specialized representations appear in 2D robotics:

1. **Differential drive robots**: Trajectories expressed in terms of left and right wheel velocities $(v_L, v_R)$ or linear and angular velocities $(v, \omega)$
2. **Ackermann steering** (car-like robots): Trajectories parameterized by speed and steering angle
3. **Holonomic robots**: Trajectories specified by velocity vectors that can point in any direction independent of orientation

### Practical Example

For a simple differential drive robot following a 2D trajectory, we might represent the robot's state at time $t$ as:

$$s(t) = (x(t), y(t), \theta(t))$$

With the kinematic model:

$$\begin{align}
\dot{x}(t) &= v(t) \cos(\theta(t)) \\
\dot{y}(t) &= v(t) \sin(\theta(t)) \\
\dot{\theta}(t) &= \omega(t)
\end{align}$$

Where $v(t)$ is the linear velocity and $\omega(t)$ is the angular velocity at time $t$.

The 2D representation is not only simpler computationally but is also sufficient for many practical robotics applications, including most ground robots, autonomous vehicles, and planar manipulators.

## Trajectory Estimation as a State Estimation Problem

### Classes of Probabilistic Sequential Models:

Probabilistic Graphical Models with *Hidden States*, estimated from *Observations*:

<div align="middle">
<img src="./assets/pgm-models.svg" alt="pgm" width="50%"/>
</div>


## Current state estimation based on past states and observations
   - **Filtering** algorithms such as:
     - Kalman filter (for linear Gaussian systems)
     - Extended Kalman filter (for nonlinear systems, robots)
     - Unscented Kalman filter (for highly nonlinear systems)
     - Particle filter (for general non-Gaussian, nonlinear systems)
   - These algorithms compute $P(x_t | y_{1:t})$ - the probability of the current state $x_t$ given all observations $y_{1:t}$ up to now ($t$)

<!--
**Next state prediction**:
   - **Prediction** algorithms including:
     - Predictive step in Kalman filtering
     - Forward models in dynamic Bayesian networks
     - State transition models in hidden Markov models
     - Forecast models in time series analysis
   - These compute $P(x_{t+1} | y_{1:t})$ - predicting the next state given observations up to now


3. **Most likely sequence of states from observation sequence**:
   - **Smoothing** and **decoding** algorithms such as:
     - Viterbi algorithm (for finding the most probable state sequence)
     - Forward-backward algorithm (for computing marginal distributions)
     - Rauch-Tung-Striebel smoother (for linear Gaussian systems)
   - These compute 
        $$arg max_{x_{1:T}} P(x_{1:T} | y_{1:T})$$
     i.e. the most likely complete state trajectory given all observations

All of these tasks can be formulated within the framework of dynamic Bayesian networks, which are temporal extensions of probabilistic graphical models that explicitly represent the evolution of random variables over time.
-->


### Principles of the Kalman Filter


The Kalman filter manages to incorporate all past measurements implicitly through recursion.

1. **State representation**: The Kalman filter maintains a sufficient statistic of all past information in the form of:
   - A state estimate (mean vector)
   - A covariance matrix (representing uncertainty)

2. **Recursive nature**: The current state estimate already contains information from all previous measurements because:
   - At each step t, the state estimate x̂(t|t) is based on x̂(t-1|t-1) and y(t)
   - But x̂(t-1|t-1) was itself based on x̂(t-2|t-2) and y(t-1)
   - And so on, all the way back to the initial state

3. **Markov property**: The Kalman filter exploits the Markov property of the system - the current state contains all the information needed to predict future states, so we don't need to store the entire history.

4. **Optimal property**: For linear Gaussian systems, this recursive estimation is provably optimal - the state estimate and covariance at time t are sufficient statistics for all measurements up to time t.

So while the Kalman filter equations only explicitly reference the previous state estimate and current observation:

```
Predict:
x̂(t|t-1) = F·x̂(t-1|t-1)
P(t|t-1) = F·P(t-1|t-1)·F' + Q

Update:
K(t) = P(t|t-1)·H'·[H·P(t|t-1)·H' + R]^(-1)
x̂(t|t) = x̂(t|t-1) + K(t)·[y(t) - H·x̂(t|t-1)]
P(t|t) = [I - K(t)·H]·P(t|t-1)
```

It's actually integrating information from all past measurements through this recursive structure. This is one of the elegant properties of the Kalman filter - it provides a compact, computationally efficient way to incorporate the entire measurement history without having to store or reprocess that history.